In [1]:
import mxnet as mx
from mxnet import nd, gluon, autograd, metric
from mxnet.gluon import loss, trainer, data, nn
import numpy as np
import multiprocessing as mp

In [2]:
BATCH_SIZE = 30
NUM_WORKERS = 1
EPOCHS = 50
if mx.context.num_gpus() > 0:
    ctx = mx.gpu()
    NUM_WORKERS = mp.cpu_count() - 3
else:
    ctx = mx.cpu()


In [3]:
train_data = mx.nd.array((np.load('data/train_data.npy')))
train_label = mx.nd.array((np.load('data/train_label.npy')))

test_data = mx.nd.array((np.load('data/test_data.npy')))
test_label = mx.nd.array((np.load('data/test_label.npy')))
(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

((3012, 17), (3012,), (754, 17), (754,))

In [4]:
for j in range(len(test_data[0,:])):
    test_data[:,j] = test_data[:,j]/test_data[:,j].mean()

for j in range(len(train_data[0,:])):
    train_data[:,j] = train_data[:,j]/train_data[:,j].mean()


In [5]:
train_data[:,7]


[0.8258843  1.9821223  1.9821223  ... 0.49553058 0.8258843  1.4865917 ]
<NDArray 3012 @cpu(0)>

In [6]:
train_dataset = data.ArrayDataset(train_data, train_label)
train_data = data.DataLoader(train_dataset, 
                             batch_size=BATCH_SIZE, 
                             shuffle=True, 
                             num_workers=NUM_WORKERS, 
                             last_batch='keep')

test_dataset = data.ArrayDataset(test_data, test_label)
test_data = data.DataLoader(test_dataset, 
                            batch_size=BATCH_SIZE, 
                            shuffle=False,
                            num_workers=NUM_WORKERS,
                            last_batch='keep')


In [7]:
train_data.__iter__().next()

[
 [[ 1.2899358   2.5310924   1.4815543   1.8967255   0.9116223   0.9165087
    0.89437896  0.8258843   1.0502092   0.5636052   1.2547013   1.6645483
    0.41818812  1.3500671   0.5692686   0.          0.        ]
  [ 0.          0.          0.          0.          0.9116223   1.4805139
    1.2521306   1.8169454   0.6301255   1.0332762   1.7028089   2.1639128
    0.6272822   1.3500671   0.5692686   0.          0.        ]
  [ 1.2899358   0.8436975   1.4815543   1.8967255   0.9116223   0.846008
    0.80494106  0.99106115  0.84016734  0.8454078   0.8962152   0.49936447
    0.6272822   0.67503357  0.5692686   0.          0.        ]
  [ 1.2899358   0.8436975   0.          0.          0.9116223   1.4805139
    0.80494106  1.1562381   1.0502092   0.8454078   1.2547013   0.6658193
    1.2545644   3.3751678   1.7078058   0.          0.        ]
  [ 1.2899358   0.8436975   2.9631085   0.          1.8232446   0.846008
    0.89437896  0.99106115  0.84016734  0.7514736   0.26886457  1.8310031
   

In [8]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(units=64, activation='tanh'))
    net.add(nn.Dropout(rate=.5))
    net.add(nn.Dense(units=3))
net

Sequential(
  (0): Dense(None -> 64, Activation(tanh))
  (1): Dropout(p = 0.5, axes=())
  (2): Dense(None -> 3, linear)
)

In [9]:
loss_fn = loss.SoftmaxCrossEntropyLoss()

In [10]:
net.initialize(ctx=ctx, force_reinit=True)

In [11]:
trainer = trainer.Trainer(net.collect_params(), 
                          optimizer='sgd', 
                          optimizer_params={'learning_rate': .001, 'momentum': 0.9, 'wd':.05512})

In [12]:
def eval_func(network, data_loader, loss_fn, metrics=[metric.RMSE()]):
    m = metric.CompositeEvalMetric()
    for mc in metrics:
        m.add(mc)
    
    m.reset()
    for i, (data, label) in enumerate(data_loader):
        output = network(data)
        loss = loss_fn(output, label)
        m.update(preds=output, labels=loss)
    return m
    

In [13]:
metric_list = [metric.RMSE(), metric.Accuracy()]
for e in range(EPOCHS):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = loss_fn(output, label)
        loss.backward()
        trainer.step(batch_size=BATCH_SIZE)
    test_eval = eval_func(network=net, data_loader=test_data, metrics=metric_list, loss_fn=loss_fn)
    train_eval = eval_func(network=net, data_loader=train_data, metrics=metric_list, loss_fn=loss_fn)
    print("EPOCH {}:".format(e))
    print("test eval: {}". format(test_eval.get()))
    print("train eval: {}". format(train_eval.get()))
    print()
    

EPOCH 0:
test eval: (['rmse', 'accuracy'], [1.0863658602875057, 0.12118193891102258])
train eval: (['rmse', 'accuracy'], [1.0863658602875057, 0.12118193891102258])

EPOCH 1:
test eval: (['rmse', 'accuracy'], [1.0746101188187551, 0.199867197875166])
train eval: (['rmse', 'accuracy'], [1.0746101188187551, 0.199867197875166])

EPOCH 2:
test eval: (['rmse', 'accuracy'], [1.109161661403014, 0.2300796812749004])
train eval: (['rmse', 'accuracy'], [1.109161661403014, 0.2300796812749004])

EPOCH 3:
test eval: (['rmse', 'accuracy'], [1.1816677775713478, 0.2450199203187251])
train eval: (['rmse', 'accuracy'], [1.1816677775713478, 0.2450199203187251])

EPOCH 4:
test eval: (['rmse', 'accuracy'], [1.259279748000721, 0.26759628154050463])
train eval: (['rmse', 'accuracy'], [1.259279748000721, 0.26759628154050463])

EPOCH 5:
test eval: (['rmse', 'accuracy'], [1.3345941257948923, 0.29415670650730413])
train eval: (['rmse', 'accuracy'], [1.3345941257948923, 0.29415670650730413])

EPOCH 6:
test eval: ([